# 合并数据_汇总：深调套利收入
读取 `data_out` 中的交易量价数据，并按照提供的公式计算 `深调套利收入`。

In [100]:
from pathlib import Path
import pandas as pd

OUTPUT_PATH = Path('../data_output/合并数据_汇总.xlsx')
SUMMARY_KEY = '合并数据_汇总'
SOURCE_KEY = '交易量价数据信息'

if not OUTPUT_PATH.exists():
    raise FileNotFoundError(f'找不到合并后的输出文件: {OUTPUT_PATH}')

data_out = pd.read_excel(OUTPUT_PATH, sheet_name=None)
sheet_order = list(data_out.keys())

if SUMMARY_KEY not in data_out:
    if SOURCE_KEY not in data_out:
        raise KeyError('在工作簿中找不到 `交易量价数据信息` 表，无法创建合并数据。')
    data_out[SUMMARY_KEY] = data_out[SOURCE_KEY].copy()
    sheet_order.append(SUMMARY_KEY)

print(f'已读取 {len(data_out)} 张表，目标: {SUMMARY_KEY}')


已读取 4 张表，目标: 合并数据_汇总


In [ ]:
# 检查河津公司的数据
hejin_data = summary_df[summary_df['公司名称'] == '河津']
print('河津公司数据:')
print(hejin_data[['公司名称', '机组名称', '省内中长期上网电量', '省间中长期上网电量', '省内中长期均价', '省间中长期均价']].to_string())

print('\n河津公司基础信息:')
print(info_df[info_df['公司名称'] == '河津'][['公司名称', '机组名称', '机组容量']].to_string())


In [ ]:
# 检查基础信息表中的机组容量
info_df = data_out['基础信息'].copy()
print('基础信息表 - 公司名称、机组名称、机组容量:')
print(info_df[['公司名称', '机组名称', '机组容量']].to_string())
print('\n同华公司的机组容量:')
print(info_df[info_df['公司名称'] == '同华'][['公司名称', '机组名称', '机组容量']])


In [ ]:
# 展示河津公司1号机组的计算过程
hejin_unit1 = summary_df[summary_df['匹配键'] == '河津1号机组'].copy()
print('河津1号机组计算过程（前5行）:')
print('=' * 80)
print(hejin_unit1[['公司名称', '机组名称', '装机容量', 'contract_power', '日前中标出力', '日前出清节点价格', '省内中长期均价']].head(5).to_string())

print('\n各项数值计算:')
for idx, row in hejin_unit1.head(5).iterrows():
    contract_power = row['contract_power']
    bid_power = row['日前中标出力']
    capacity = row['装机容量']
    spot_price = row['日前出清节点价格']
    contract_price = row['省内中长期均价']
    coefficient = 1.0
    
    limit = contract_power * 4 / capacity * coefficient
    spread = (limit - bid_power) * (contract_price - spot_price) / 4
    
    print(f'行{idx}:')
    print(f'  contract_power={contract_power:.2f}, bid_power={bid_power:.2f}, capacity={capacity:.0f}')
    print(f'  limit={limit:.2f}, spot_price={spot_price:.2f}, contract_price={contract_price:.2f}')
    print(f'  spread={spread:.4f}')
    print(f'  深调套利收入={row["深调套利收入"]:.4f}')
    print()


In [106]:
summary_df = data_out[SUMMARY_KEY].copy()
info_df = data_out['基础信息'].copy()

required_columns = ['日前中标出力', '省内中长期上网电量', '日前出清节点价格', '省内中长期均价', '日期', '公司名称', '日内实际出力']
missing_columns = [col for col in required_columns if col not in summary_df.columns]
if missing_columns:
    raise ValueError(f'{SUMMARY_KEY} 缺少以下列: {missing_columns}')

if '机组容量' not in info_df.columns or '机组名称' not in info_df.columns:
    raise ValueError(f'基础信息 缺少必需列: 机组容量 或 机组名称')

# 日期筛选
start_date = pd.to_datetime('2026-01-27')  # 设置起始日期
end_date = pd.to_datetime('2026-01-27')    # 设置结束日期

summary_df['日期'] = pd.to_datetime(summary_df['日期'])
summary_df = summary_df[(summary_df['日期'] >= start_date) & (summary_df['日期'] <= end_date)]

# 新增筛选：日前出清节点价格 [0, 200]
summary_df = summary_df[(summary_df['日前出清节点价格'] >= 0) & (summary_df['日前出清节点价格'] <= 200)]

print(f'筛选后剩余数据: {len(summary_df)} 行')

def to_numeric(series):
    return pd.to_numeric(series, errors='coerce')

bid_power = to_numeric(summary_df['日前中标出力'])
intra_prov_contact_power = to_numeric(summary_df['省内中长期上网电量'])
intra_prov_contact_avg_price = to_numeric(summary_df['省内中长期均价'])

# 检查是否存在省间数据
has_inter = '省间中长期上网电量' in summary_df.columns and '省间中长期均价' in summary_df.columns

if has_inter:
    inter_prov_contact_power = to_numeric(summary_df['省间中长期上网电量']).fillna(0)
    inter_prov_contact_avg_price = to_numeric(summary_df['省间中长期均价']).fillna(0)
    contract_power = intra_prov_contact_power + inter_prov_contact_power
    # 计算加权平均价格
    contract_price = (intra_prov_contact_power * intra_prov_contact_avg_price + 
                       inter_prov_contact_power * inter_prov_contact_avg_price) / contract_power
else:
    inter_prov_contact_power = 0
    contract_power = intra_prov_contact_power
    contract_price = intra_prov_contact_avg_price

spot_price = to_numeric(summary_df['日前出清节点价格'])

# 使用公司名称+机组名称作为联合关键字匹配装机容量
info_df['匹配键'] = info_df['公司名称'] + info_df['机组名称']
capacity_mapping = info_df.set_index('匹配键')['机组容量'].to_dict()
summary_df['匹配键'] = summary_df['公司名称'] + summary_df['机组名称']
summary_df['装机容量'] = summary_df['匹配键'].map(capacity_mapping)
summary_df['装机容量'] = to_numeric(summary_df['装机容量'])

# 新增系数（可根据需要调整）
coefficient = 660  # 深调系数

# 计算深调套利收入
condition = (bid_power < contract_power * 4) & (spot_price < contract_price)
spread = (contract_power * 4 - bid_power) * coefficient / summary_df['装机容量'] * (contract_price - spot_price) / 4
summary_df['深调套利收入'] = spread.where(condition, 0).fillna(0)
summary_df['contract_power'] = contract_power

# 计算每台机组的深调平均负荷
summary_df['单台深调平均负荷'] = to_numeric(summary_df['日内实际出力']) / summary_df['装机容量']

# 先按公司名称+机组名称计算单台机组指标
unit_result_df = summary_df.groupby('匹配键').agg(
    公司名称=('公司名称', 'first'),
    日前低价时长_小时_=('日前出清节点价格', 'count'),
    现货价格_=('日前出清节点价格', 'mean'),
    深调平均负荷_=('单台深调平均负荷', 'mean'),
    中长期平均持仓_=('contract_power', 'mean'),
    深调套利_元_=('深调套利收入', 'sum')
).reset_index()

# 再按公司名称汇总（对两台机组的指标求平均）
result_df = unit_result_df.groupby('公司名称').agg(
    日前低价时长_小时_=('日前低价时长_小时_', 'mean'),
    现货价格_=('现货价格_', 'mean'),
    深调平均负荷_=('深调平均负荷_', 'mean'),
    中长期平均持仓_=('中长期平均持仓_', 'mean'),
    深调套利_元_=('深调套利_元_', 'sum')
).reset_index()

# 日前低价时长除以4转换为小时数
result_df['日前低价时长_小时_'] = result_df['日前低价时长_小时_'] / 4

result_df.columns = ['单位', '日前低价时长（小时）', '现货价格', '深调平均负荷', '中长期平均持仓', '深调套利（元）']
result_df

筛选后剩余数据: 408 行


,单位,日前低价时长（小时）,现货价格,深调平均负荷,中长期平均持仓,深调套利（元）
0,临汾,4.25,38.764706,0.597816,43.220353,24678.459773
1,侯马,4.25,38.764706,0.532989,39.658176,6088.422560
2,同华,4.25,38.764706,0.401894,115.474882,250424.393850
3,同承,4.25,38.764706,0.575903,6.688838,5468.061324
4,同达,4.25,38.764706,0.393733,67.711412,405962.741362
5,塔山,4.25,38.764706,0.382312,83.878059,105862.794213
6,河津,4.25,38.764706,0.319128,61.202971,288207.779998
7,王坪,8.50,38.764706,0.496288,31.602294,211329.786888
8,蒲洲,4.25,38.764706,0.483861,56.535471,120636.416055
9,阳高,4.25,38.764706,0.286681,48.902235,157231.496683
